<a href="https://colab.research.google.com/github/LeonMilosevic/covid_19_analysis/blob/main/covid_19_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Covid-19 EDA**

## Introduction



1. We will explore how South Korea dealt with pandemic compared to Other countries. 
2. We will explore specifically how South Korea battled with the pandemic.
3. We will create a model to distinguish between released and isolated patients in South Korea. 

### Datasets used:


*   https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset
*   https://www.kaggle.com/kimjihoo/coronavirusdataset/
*   https://www.kaggle.com/dumbgeek/countries-dataset-2020
*   https://www.kaggle.com/eng0mohamed0nabil/population-by-country-2020

In [37]:
def get_max_value(series):
  # reindex the series
  reseted = series.reset_index(drop=True)

  # get max id and value
  maxid = reseted.idxmax()
  max = reseted.max()
  # if maxid is not the last value and the max value is not equal to the last value then loop over the remaining values and add them to max variable
  if(maxid != len(reseted) -1 and reseted[maxid] != reseted.iloc[-1]):
    # adding + 1 to the id since we want to skip maxid
    for i, val in enumerate(reseted[maxid + 1:], start=maxid + 1):
      max += val
      
  return max

In [2]:
import pandas as pd

# all countries cases
countries_data = pd.read_csv('./covid_19_data.csv')

# healthcare of countries
countries_healthcare = pd.read_csv('./healthcare.csv')

South Korea has one of the best health care index in the world, we will compare it with countries that have similar healthcare, +-5

In [3]:
south_korea_healthcare = countries_healthcare[countries_healthcare['Country'] == 'South Korea']['Health Care Index'][1]

countries_strong_healthcare = countries_healthcare.loc[
                                                       (countries_healthcare['Health Care Index'] >= south_korea_healthcare - 5) & 
                                                       (countries_healthcare['Health Care Index'] <= south_korea_healthcare + 5)
                                                       ]

In [46]:
countries_cases_healthcare = pd.DataFrame(columns=countries_data.columns)

# for i in range(len(countries_strong_healthcare.Country)):
#   country = countries_strong_healthcare.Country[i]
#   country_cases = countries_data[countries_data['Country/Region'] == country]

#   print(country_cases)
#   # countries_cases_healthcare.loc[i] = country_cases

countries_data[countries_data['Country/Region'] == 'Denmark'].reset_index(drop=True)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,2563,02/27/2020,NaN,Denmark,2020-02-27T16:23:03,1.0,0.0,0.0
1,2673,02/28/2020,NaN,Denmark,2020-02-27T16:23:03,1.0,0.0,0.0
2,2770,02/29/2020,NaN,Denmark,2020-02-29T18:03:05,3.0,0.0,0.0
3,2889,03/01/2020,NaN,Denmark,2020-03-01T23:33:03,4.0,0.0,0.0
4,3019,03/02/2020,NaN,Denmark,2020-03-01T23:33:03,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...
813,171271,12/05/2020,Faroe Islands,Denmark,2020-12-06 05:26:18,506.0,0.0,500.0
814,171296,12/05/2020,Greenland,Denmark,2020-12-06 05:26:18,18.0,0.0,18.0
815,171760,12/06/2020,NaN,Denmark,2020-12-07 05:26:14,90603.0,885.0,70876.0
816,172031,12/06/2020,Faroe Islands,Denmark,2020-12-07 05:26:14,510.0,0.0,501.0
